# 06. CODE REPRESENTATION

1. CodeBERT
2. GraphCodeBERT
3. UniXCoder
4. Exercise
5. References

# 1. CodeBERT

[CodeBERT](https://arxiv.org/abs/2002.08155v4) is a bimodal model for programming and natural language. CodeBERT outputs general-purpose embeddings. Embeddings can be used in tasks such as code search, code summarization, etc. The model was developed at Microsoft in 2020.
![](./res/06_codebert_paper.png)

The CodeBERT model is based on [BERT](https://arxiv.org/abs/1810.04805)(Google, 2018). More precisely, [RoBERTa](https://arxiv.org/abs/1907.11692) (Meta, 2019, the difference is in the mask generation procedure).

#### CodeBERT на Huggingface

In [ ]:
import pprint
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
model = AutoModel.from_pretrained('microsoft/codebert-base')

In [ ]:
nl_tokens=tokenizer.tokenize('return maximum value')
print(nl_tokens)

In [ ]:
code_tokens=tokenizer.tokenize('def max(a,b): if a>b: return a else return b')
print(code_tokens)

In [ ]:
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
print(tokens)

In [ ]:
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
print(tokens_ids)

In [ ]:
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]

In [ ]:
context_embeddings.shape

In [ ]:
print(context_embeddings)

#### Pre-training

CodeBERT is a bimodal model. This means that pairs of the form (NL, PL) can be fed to the input.
In this case, the pair is transformed into a sequence of tokens.

To do this, each element of the pair: the NL and PL part, is transformed into a sequence of tokens (segment) using the tokenizer ([Byte Pair Encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding), BPE).

At the pre-training stage, the concatenation of two segments with a special separator token is fed to the input:

$$[\texttt{CLS}], w_1, w_2, \ldots, w_n, [\texttt{SEP}], c_1, c_2, \ldots, c_m, [\texttt{EOS}].$$

- first segment: natural language text
- second segment: source code

Pretraining is done on two tasks:
- MLM (masked language modeling), BERT, Devlin et al. (2018): token recovery, bimodal data
- RTD (replaced token detection), [ELECTRA, Clark et al. (2020)](https://arxiv.org/abs/2003.10555): token replacement detection, unimodal data.

$[\texttt{CLS}]$ --- special token for obtaining aggregated embedding (can be used, for example, in classification)

At the output we have:
- context vector representation of each token (for natural language and code);
- representation for the token $[\texttt{CLS}]$.

#### Pretrain data

The CodeBERT model is trained on both *bimodal* data (parallel text and code pairs) and *unimodal* data:

- Github repositories
- bimodal pair --- a function and its documentation (approximately 2 million)
- unimodal data --- a function without its documentation pair (approximately 6 million)
- 6 programming languages: Python, Java, JavaScript, PHP, Ruby and Go.

![example of a bimodal pair](./res/06_codebert_bimodal.png)

NL part --- only the text part (highlighted in red) of the documentation (dotted outline).

#### MLM Task

Let $x = \{w, c\}$ be an input NL-PL pair, where $w$ is a sequence of NL tokens and $c$ is a sequence of PL tokens.

First, we select a random set of positions for NL and PL to be masked --- $m^w$ and $m^c$, respectively. Then, we replace the selected positions with a special token $[\texttt{MASK}]$.

As in [Devlin et al. (2018)](https://arxiv.org/abs/1810.04805), 15% of tokens from $x$ are masked --- if a token is selected, then with probability 0.8 it is replaced with $[\texttt{MASK}]$, with probability 0.1 it is replaced with a random token, with probability 0.1 it remains the same).

- for $i \in \{1, \ldots, |w|\}$ we have $m^w_i ∼ \texttt{unif}\{1, |w|\}$ --- which NL tokens we want to mask
- for $i \in \{1, \ldots, |c|\}$ we have $m^c_m ∼ \texttt{unif}\{1, |c|\}$ --- which PL tokens we want to mask
- $w^{masked} = \texttt{REPLACE}(w, m^w, [\texttt{MASK}])$
- $c^{masked} = \texttt{REPLACE}(c, m^c, [\texttt{MASK}])$
- $x = [w;c]$ --- concatenate

In MLM, the task is to predict the tokens that have been masked. For this purpose, the following loss function is used:

$$L_{MLM}(\theta) = \sum_{i \in m^w \cup m^c} -\log p (x_i~|~w^{masked}, c^{masked}).$$

In [ ]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained('microsoft/codebert-base-mlm')
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

In [ ]:
code = """
if x is not <mask>:
    x += 1"
"""
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer, device=device)

outputs = fill_mask(code)
pp.pprint(outputs)

#### Задача RTD

Изначально, задача RTD (replaced token detection) была использована для предобучения моделей естественного языка [ELECTRA](https://arxiv.org/abs/2003.10555). Для модели CodeBERT эта задача была адаптирована для кода.
Используются как *бимодальные*, так и *унимодальные* данные. В частности, здесь есть два генератора данных:
- NL-генератор $p^{G_w}$ и
- PL-генератор $p^{G_c}$.

Оба генератора нужны для создания правдоподобных замен выбранных токенов.

![](./res/06_rtd_electra.png)

Генератор --- произвольная языковая модель, генерирующая распределение на токенах. Обычно используют небольшую  модель, обучающуюся совместно с генератором.

Более формально:
- для $i \in m^w$ имеем $\hat{w}_i ∼ p^{G_w} (w_i~|~w_{masked})$ --- какие токены и на какие хотим заменить
- для $i \in m^c$ имеем $\hat{c}_i ∼ p^{G_c} (c_i~|~c_{masked})$ --- аналогично для кода
- $w^{corrupt} = \texttt{REPLACE}(w, m^w , \hat{w})$
- $c^{corrupt} = \texttt{REPLACE}(c, m^c , \hat{c})$
- $x^{corrupt} = [w^{corrupt};c^{corrupt}]$ --- конкатенируем 

#### RTD Task

The RTD (replaced token detection) task was originally used to pretrain natural language models [ELECTRA](https://arxiv.org/abs/2003.10555). For the CodeBERT model, this task was adapted for code.
Both *bimodal* and *unimodal* data are used. In particular, there are two data generators:
- NL-generator $p^{G_w}$ and
- PL-generator $p^{G_c}$.

Both generators are needed to create plausible replacements for the selected tokens.

![](./res/06_rtd_electra.png)

Generator --- an arbitrary language model that generates a distribution on tokens. Usually, a small model is used, trained jointly with the generator.

More formally:
- for $i \in m^w$ we have $\hat{w}_i ∼ p^{G_w} (w_i~|~w_{masked})$ --- which tokens and with which we want to replace
- for $i \in m^c$ we have $\hat{c}_i ∼ p^{G_c} (c_i~|~c_{masked})$ --- similarly for the code
- $w^{corrupt} = \texttt{REPLACE}(w, m^w , \hat{w})$
- $c^{corrupt} = \texttt{REPLACE}(c, m^c , \hat{c})$
- $x^{corrupt} = [w^{corrupt};c^{corrupt}]$ --- concatenation

The loss function for the RTD task is:

$$L_{RTD}(\theta) = \sum_{i=1}^{|w|+|c|}(\delta(i) \log p^{D} (x^{corrupt}, i) + (1-\delta(i))(1-\log p^{D}(x^{corrupt}, i))),$$

where
- $\theta$ are parameters,
- $p^{D}$ is a discriminator that predicts the probability that the $i$-th token is original,
- $\delta(i)$ is an indicator function: $\delta(i) = 1$ if $x^{corrupt}_i = x_i$; otherwise, $\delta(i) = 0.$

There are many different ways to implement generators. CodeBERT uses two $n$-gram language models (NL, PL) (Jurafsky, 2000). The models were trained on the corresponding unimodal data.

#### MLM+RTD task

The resulting task looks like this:
$$ \min_{\theta}( L_{MLM}(\theta) + L_{RTD}(\theta) ).$$

#### Implementation

- https://github.com/microsoft/CodeBERT
- https://huggingface.co/microsoft/graphcodebert-base

#### Downstream tasks

- natural language code search
- code documentation generation
- clone detection etc.

# 2. GraphCodeBERT

A pre-trained model for code that considers the **inherent structure** of code

The model uses **data flow** in the pre-training stage.

![](res/06_graphcodebert_paper.png)

#### DATA FLOW

Data flow is a graph that represents **dependency relation between variables**, in which
- nodes represent variables and
- edges represent where the value of each variable comes from.

Unlike AST, data flow **is same under different abstract grammars** for the same source code.
Such code structure provides crucial code semantic information for code understanding.


> Example:
>
> ![overview](./res/06_graphcodebert_dataflow.png)
>
> There are four variables with same name (i.e. $x^3$, $x^7$, $x^9$ and $x^{11}$) but with different semantic. The graph in the figure shows dependency relation between these variables and supports $x^{11}$ to pay more attention to $x^7$ and $x^9$ instead of $x^3$.

Data flow construction algorithm:

1. source code $C = \{c_1, c_2, ..., c_n \}$
2. parse the code into an abstract syntax tree (AST) by a standard compiler tool
3. the AST includes syntax information of the code and terminals (leaves) are used to identify the variable sequence, denoted as $V = \{v_1, v_2, ..., v_k\}$
4. take each variable as a node of the graph and an direct edge $e = (v_i, v_j)$ from $v_i$ to $v_j$ refers that the value of $j$-th variable comes from $i$-th variable
5. denote the set of directed edges as $E = \{e_1, e_2 , ..., e_l\}$ and the graph $G(C) = (V, E)$ is data flow used to represent dependency relation between variables of the source code $C$

#### MODEL ARCHITECTURE

![overview](./res/06_graphcodebert_overview.png)

- Text --- comment to source code, $W = \{w_1, w_2, ..., w_m\}$
- Source Code $C = \{c_1, c_2, ..., c_n\}$
- Variable Sequence $V = \{v_1, v_2, ..., v_k\}$, where $G(C) = (V, E)$ is the corresponding data flow with $E = \{e_1, e_2, ..., e_l\}$ is a set of direct edges that represent where the value of each variable comes from

Input:
- We concatenate the comment, source code and the set of variables as the sequence input $X = \{[CLS], W, [SEP], C, [SEP], V \}$, where $[CLS]$ is a special token in front of three segments and $[SEP]$ is a special symbol to split two kinds of data types.

GraphCodeBERT takes the sequence $X$ as the input and then converts the sequence into input vectors $H^0$. For each token, its input vector is constructed by summing the corresponding token and position embeddings. We use a special position embedding for all variables to indicate that they are nodes of data flow.

The model applies $N$ transformer layers over the input vectors to produce contextual representations $H^n = \texttt{transformer}_n(H^{n−1})$, $n \in [1, N]$. Each transformer layer contains an architecturally identical transformer that applies a multi-headed self-attention operation (Vaswani et al., 2017) followed by a feed forward layer over the input $H^{n-1}$ in the $n$-th layer.

$$G^n = \texttt{LN} (\texttt{MultiAttn} (H^{n-1}) + H^{n-1})$$
$$H^n = \texttt{LN} (\texttt{FFN} (G^n) + G^n)$$

where $\texttt{MultiAttn}$ is a multi-headed self-attention mechanism, $\texttt{FFN}$ is a two layers feed forward network, and $\texttt{LN}$ represents a layer normalization operation. For the $n$-th transformer layer, the output $\hat{G}^n$ of a multi-headed self-attention is computed via:

$$Q_i = H^{n-1}W^Q_i$$
$$K_i = H^{n-1}W^K_i$$
$$V_i = H^{n-1}W^V_i$$
$$\texttt{head}_i = \texttt{softmax}(\frac{Q_iK^T_i}{\sqrt{d_k}} + M)V_i ~~~~~ (*)$$
$$\hat{G}^n = [\texttt{head}_1 ; ...; \texttt{head}_u]W^O_n$$

where the previous layer’s output $H^{n-1} \in R^{|X|} \times R^{d_h}$ is linearly projected to a triplet of queries, keys and values using model parameters $W^Q_i, W^K_i, W^V_i \in R^{d_h} \times R^{d_k}$, respectively. $u$ is the number of heads, $d_k$ is the dimension of a head, and $W^O_n \in R^{d_h} \times R^{d_h}$ is the model parameters.


$M \in R^{|X|} \times R^{|X|}$ is a mask matrix, where $M_{ij}$ is $0$ if $i$-th token is allowed to attend $j$-th token otherwise $-\infty$.

#### GRAPH-GUIDED MASKED ATTENTION

To incorporate the graph structure into Transformer, we define a graph-guided masked attention function to filter out irrelevant signals.

The attention masking function could avoid the key $k_i$ attended by the query $q_j$ by adding the attention score $q^T_jk_i$ an **infinitely negative value** so that the attention weight becomes zero after using a softmax function.

To represent dependency relation between variables, a node-query $q_{v_i}$ is allowed to attend to a node-key $k_{v_j}$ if there is a direct edge from the node $v_j$ to the node $v_i$ (i.e. $(v_j, v_i) \in E$) or they are the same node (i.e. $i=j$). Otherwise, the attention is masked by adding an infinitely negative value into the attention score.

To represent the relation between source code tokens and nodes of the data flow, we first define a set $E'$, where $(v_i, c_j) \in E'$ if the variable $v_i$ is identified from the source code token $c_j$. We then allow the node $q_{v_i}$ and code $k_{c_j}$ attend each other if and only if $(v_i, c_j) \in E'$.

More formally, we use the following graph-guided masked attention matrix as the mask matrix $M$ in the equation ( * ):

$M_{ij} = 0$ if $q_i \in \{[CLS], [SEP]\}$ or $q_i, k_j \in W \cup C$ or $(q_i, k_j) \in E \cup E'$;

$M_{ij} = - \infty$ otherwise.

#### PRE-TRAINING TASKS

- masked language modeling
- data flow edge prediction
- variable-alignment across source code and data flow

**Masked Language Modeling**

- sample randomly 15% of the tokens from the source code and paired comment
- replace them with a [MASK] token 80% of the time, with a random token 10% of the time, and leave them unchanged 10% of the time

**Edge Prediction**


- randomly sample 20% of nodes $V_s$ in data flow, mask direct edges connecting these sampled nodes by add an infinitely negative value in the mask matrix, and then predict these masked edges $E_{mask}$

![](./res/06_data_flow_graph.png)

Taking the variable $x^{11}$ in Figure for an example, we first mask edges $(x^7, x^{11})$ and $(x^9, x^{11})$ in the graph and then let the model to predict these edges.

Formally, the pre-training objective of the task is calculated as Equation ( ** ), where $E_c = V_s \times V \cup V \times V_s$ is a set of candidates for edge prediction, $\delta(e_{ij} \in E)$ is $1$ if $(v_i, v_j) \in E$ otherwise $0$, and the probability $p_{e_{ij}}$ of **existing an edge from $i$-th to $j$-th node is calculated by dot product following a sigmoid function** using representations of two nodes from GraphCodeBERT. To balance positive-negative ratio of examples, we sample negative and positive samples with the same number for $E_c$.

$$loss_{EdgePred} = - \sum_{e_{ij}\in E_c} [\delta(e_{ij} \in E_{mask})\log(p_{e_{ij}}) + (1-\delta(e_{ij} \in E_{mask})) \log(1-p_{e_{ij}})]~~~~~(**)$$

**Node Alignment**

The motivation is to encourage the model to align variables and source code according to data flow.

- predict edges between code tokens and nodes.

Taking Figure for an example, we first mask edges between the variable $x^{11}$ in data flow and code tokens, and then predict which code token the variable $x^{11}$ in data flow is identified from. As we can see, the model could predict that the variable $x^{11}$ is identified form the variable $x$ in the expression “return $x$” according to data flow information (i.e. the value of $x^{11}$ comes from $x^7$ or $x^9$).

![overview](./res/06_graphcodebert_node_task.png)

Specially, we randomly sample 20% nodes $V'_s$ in the graph, mask edges between code tokens and sampled nodes, and then predict masked edges $E'_{mask}$. The pre-training objective of this task is similar to Equation ( ** ), where $E'_c = V'_s \times C$ is a set of candidates for node alignment. Similarly, we also sample negative and positive samples with the same number for $E'_c$.

$$loss_{NodeAlign} = - \sum_{e_{ij}\in E'_c} [\delta(e_{ij} \in E'_{mask})\log(p_{e_{ij}}) + (1-\delta(e_{ij} \in E'_{mask})) \log(1-p_{e_{ij}})]$$

# 3. UniXCoder

![](res/06_unixcoder_paper.png)

A unified cross-modal pre-trained model for programming language.

Tasks:
- code-related understanding
- code-related generation

![](res/06_unixcoder_architecture.png)

https://github.com/microsoft/CodeBERT/tree/master/UniXcoder

# 4. Exercise

Explore the possibility of using CodeBERT or GraphCodeBERT for the clone detection task.
For example, you can use the [BigCloneBench](https://github.com/clonebench/BigCloneBench) dataset

# 5. References

- [Kanade et al - CuBERT: Learning and Evaluating Contextual Embedding of Source Code](https://arxiv.org/abs/2001.00059)
- [Feng et al - CodeBERT: A Pre-trained Model for Programming and Natural Languages 2020](https://arxiv.org/abs/2002.08155)
- [Karampatsis Sutton - SCELMo: Source Code Embeddings from Language Models 2020](https://arxiv.org/abs/2004.13214)
- [Jain et al - Contrastive Code Representation Learning 2020](https://arxiv.org/abs/2007.04973)
- [Guo et al - GraphCodeBERT: Pre-training Code Representations with Data Flow 2020](https://arxiv.org/abs/2009.08366)
- [Chirkova Torshin - An Empirical Study of Transformers for Source Code 2020](https://arxiv.org/abs/2010.07987)
- [Gu et al - Multimodal representation for neural code search 2022](https://arxiv.org/abs/2107.00992)
- [Wang et al - CodeT5: Identifier-aware Unified Pre-trained Encoder-Decoder Models for Code Understanding and Generation](https://arxiv.org/abs/2109.00859)
- [Guo et al - UniXcoder: Unified Cross-Modal Pre-training for Code Representation](https://arxiv.org/abs/2203.03850)